# Tuning curve changes for Figure 1c in Loback et al.

##### From driscoll et al

Dynamic Reorganization of Neuronal Activity Patterns in Parietal Cortex 

Laura N. Driscoll, Noah L. Pettit, Matthias Minderer, Selmaan N. Chettih, Christopher D. Harve

*The changes across time resulted from cells with a peak of activity on one day either losing that peak or having a shift in the peak’s location on subsequent days, both of which increased in likelihood with time from when a peak was identified. The loss of peaks of activity was offset by an approximately constant rate at which cells initially lacking a peak of activity gained an activity peak, resulting in a consistent fraction of active cells with activity peaks on each day. Together, these results indicate that activity patterns changed over time, with major reorganization occurring over several weeks.*

### Prepare workspace

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Add local scripts to path
import os,sys
sys.path.insert(0,os.path.abspath("./"))

# Change path to match location of PPC datasets
# Datasets are available from the Harvey lab
import ppc_data_loader
ppc_data_loader.path = '/home/mrule/Workspace2/PPC_data/'

# Import PPC libraries
from ppc_data_loader   import *
from ppc_trial         import *

(In some versions of Jupyter, we need to configure Matplotlib in a separate cell.)

In [ ]:
%matplotlib inline

from cycler import cycler
matplotlib.rcParams['axes.prop_cycle'] = cycler(color=[BLACK,RUST,TURQUOISE,OCHRE,AZURE])
matplotlib.rcParams['figure.dpi']=150
figure() # Check that Matplotlib really did load (bug workaround?)

print('Matplotlib configured')

# Neural tuning curve drift plots

This generates the black and white tuning curve drift plots seen in Loback et al. Figure 1C. 

In [ ]:
def plot_unit_summary_figure(animal,unit,
                            TIMERES = 400,
                            LOWF    = 0.03,
                            HIGHF   = 0.3
                             ):
    
    sessions    = array(get_session_ids(animal))
    valid       = array([good_units(animal,s) for s in sessions])
    ok_sessions = sessions[find(valid[:,unit])]
    FS          = get_FS(animal,sessions[0])
    
    Y = [get_smoothed_dFF(animal,s,unit,LOWF,HIGHF) for s in ok_sessions]
    Z = [align_trials(animal,[s],array([[y]]),
                      PSTIMERES=TIMERES)[0][...,0] for (s,y) in zip(ok_sessions,Y)]
    
    figure(figsize=(2.125,2.125))
    maxsession = max(ok_sessions)
    minsession = min(ok_sessions)
    srange     = maxsession-minsession+1
    NSESSIONS  = len(ok_sessions)
    
    # Normalized average of location-triggered activity
    # (x-<x>)/<x> = x/<x>-1 ≃ log(x/<x>) = log(x)-log(<x>)
    tuning = {}
    for i,(session,z) in enumerate(zip(ok_sessions,Z)):
        tuning[session] = slog(maximum(1,bandpass_filter(exp(mean(z[:,:],axis=0)),fb=10,Fs=TIMERES)))
    
    imagegrid = zeros((srange,TIMERES))
    smap      = zeros(srange)
    
    for i,s in enumerate(arange(minsession,maxsession+1)):
        # If a session is missing, use nearest available sessions
        nearest = ok_sessions[argmin(abs(s-ok_sessions))]
        imagegrid[i,:] = tuning[nearest]
        smap[i] = nearest
    
    vmin = 0
    vmax = int(ceil(10*np.percentile(imagegrid.ravel(),99.9)))/10
    imshow(imagegrid,
           vmin=vmin,
           vmax=vmax,
           aspect='auto',
           cmap='bone_r',
           origin='lower')
    noxyaxes()
    
    tics = [mean(find(smap==i)) for i in ok_sessions]
    yticks(tics,['%s'%minsession]+['']*(NSESSIONS-2)+['%s'%maxsession])
    xticks(xlim(),['0%','100%'])
    force_aspect()
    ylabel('Day #')
    xlabel('Task location')
    fudgey()
    fudgex()
    title('Subject %d Neuron %d'%(unit,animal),fontsize=10)
    ax = good_colorbar(vmin,vmax,'bone_r','Mean log-rate\n(normalized)',fontsize=9)
print('`plot_unit_summary_figure` function defined')

## Select neurons

Some examples for drift, stability, and more bizzare behavior

In [ ]:
toplot={
    'Advancing':[(3,347),(3,1299),(4,124),(4,134),(4,475),(4,496)],
    'Noisy'    :[(1,13),(3,337)],
    'Other'    :[(1,1115),(3,418),(3,460),(3,954),(3,978)],
    'Relocate' :[(3,19),(4,3),(4,473)],
    'Stable'   :[(1,1263),(3,469),(3,522),(3,1191),(4,138),(4,155),(5,499)]
}

In [ ]:
for category,pairs in toplot.items():
    for animal,unit in pairs:
        print(category,animal,unit)
        fn = 'unit_drift_summary_subject_%02d_unit_%d'%(animal,unit)
        if not fn in os.listdir('.'):
            plot_unit_summary_figure(animal,unit)
            savefigure(fn)
            release_files(clear_cache=True)
